In [24]:
from DGLim.data import *
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from DGLim.hyperparameter import hp
import warnings


%reload_ext autoreload
%autoreload 2

In [ ]:
class Train:
    def __init__(self, model=DEepGriffinLim(),save_model=False, load_model=False):
        self.model = model
        self.criterion = nn.L1Loss(reduction='none')
        self.optimizer = optim.Adam(self.model.parameters(),lr=hp.learning_rate, weight_decay=hp.weight_decay)
        self.save_model = save_model
        self.load_model = load_model


    
    def save_model(self, model,loss,epoch, save_path=hp.save_path):
        torch.save({
            'epoch': epoch,
            'model_state_dict': self.model.state_dict(),
            'loss': loss,
            }, save_path)

    def load_model(self, model, load_path=hp.load_path):
        checkpoint = torch.load(load_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        epoch = checkpoint['epoch']
        loss = checkpoint['loss']
        return model, epoch, loss


    def calc_loss(self, out_blocks: Tensor, y: Tensor, T_ys: Sequence[int]) -> Tensor:
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            loss_no_red = self.criterion(out_blocks, y.unsqueeze(1))
        loss_blocks = torch.zeros(out_blocks.shape[1], device=y.device)
        for T, loss_batch in zip(T_ys, loss_no_red):
            loss_blocks += torch.mean(loss_batch[..., :T], dim=(1, 2, 3))

        if len(loss_blocks) == 1:
            loss = loss_blocks.squeeze()
        else:
            loss = loss_blocks @ self.loss_weight
        return loss
        